In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName(
    "PySpark_Dataframe_Insert_into_Spark_table"
).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/22 15:13:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/22 15:13:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/02/22 15:13:54 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/02/22 15:13:54 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [18]:
df = (
    spark.read.option("sep", ",")
    .option("inferSchema", "true")
    .option("header", "true")
    .csv("Baby_Names.csv")
)
df.show()

+----+----------+-----------+---+-----+
|Year|First Name|     County|Sex|Count|
+----+----------+-----------+---+-----+
|2007|      ZOEY|      KINGS|  F|   11|
|2007|      ZOEY|    SUFFOLK|  F|    6|
|2007|      ZOEY|     MONROE|  F|    6|
|2007|      ZOEY|       ERIE|  F|    9|
|2007|       ZOE|     ULSTER|  F|    5|
|2007|       ZOE|WESTCHESTER|  F|   24|
|2007|       ZOE|      BRONX|  F|   13|
|2007|       ZOE|   NEW YORK|  F|   55|
|2007|       ZOE|     NASSAU|  F|   15|
|2007|       ZOE|       ERIE|  F|    6|
|2007|       ZOE|    SUFFOLK|  F|   14|
|2007|       ZOE|      KINGS|  F|   34|
|2007|       ZOE|     MONROE|  F|    9|
|2007|       ZOE|     QUEENS|  F|   26|
|2007|       ZOE|     ALBANY|  F|    5|
|2007|     ZISSY|   ROCKLAND|  F|    5|
|2007|     ZISSY|      KINGS|  F|   27|
|2007|      ZION|      KINGS|  M|   15|
|2007|      ZION|      BRONX|  M|   14|
|2007|       ZEV|   ROCKLAND|  M|    6|
+----+----------+-----------+---+-----+
only showing top 20 rows



In [12]:
df.rdd.getNumPartitions()

1

In [4]:
from pyspark.sql.functions import spark_partition_id

In [5]:
df1 = df.withColumn("partitionId", spark_partition_id())
df1.show(truncate=False)

+----+----------+-----------+---+-----+-----------+
|Year|First Name|County     |Sex|Count|partitionId|
+----+----------+-----------+---+-----+-----------+
|2007|ZOEY      |KINGS      |F  |11   |0          |
|2007|ZOEY      |SUFFOLK    |F  |6    |0          |
|2007|ZOEY      |MONROE     |F  |6    |0          |
|2007|ZOEY      |ERIE       |F  |9    |0          |
|2007|ZOE       |ULSTER     |F  |5    |0          |
|2007|ZOE       |WESTCHESTER|F  |24   |0          |
|2007|ZOE       |BRONX      |F  |13   |0          |
|2007|ZOE       |NEW YORK   |F  |55   |0          |
|2007|ZOE       |NASSAU     |F  |15   |0          |
|2007|ZOE       |ERIE       |F  |6    |0          |
|2007|ZOE       |SUFFOLK    |F  |14   |0          |
|2007|ZOE       |KINGS      |F  |34   |0          |
|2007|ZOE       |MONROE     |F  |9    |0          |
|2007|ZOE       |QUEENS     |F  |26   |0          |
|2007|ZOE       |ALBANY     |F  |5    |0          |
|2007|ZISSY     |ROCKLAND   |F  |5    |0          |
|2007|ZISSY 

In [13]:
df2 = df.repartition(10).withColumn("partitionId", spark_partition_id())
df.show(truncate=False)

+----+----------+-----------+---+-----+
|Year|First Name|County     |Sex|Count|
+----+----------+-----------+---+-----+
|2007|ZOEY      |KINGS      |F  |11   |
|2007|ZOEY      |SUFFOLK    |F  |6    |
|2007|ZOEY      |MONROE     |F  |6    |
|2007|ZOEY      |ERIE       |F  |9    |
|2007|ZOE       |ULSTER     |F  |5    |
|2007|ZOE       |WESTCHESTER|F  |24   |
|2007|ZOE       |BRONX      |F  |13   |
|2007|ZOE       |NEW YORK   |F  |55   |
|2007|ZOE       |NASSAU     |F  |15   |
|2007|ZOE       |ERIE       |F  |6    |
|2007|ZOE       |SUFFOLK    |F  |14   |
|2007|ZOE       |KINGS      |F  |34   |
|2007|ZOE       |MONROE     |F  |9    |
|2007|ZOE       |QUEENS     |F  |26   |
|2007|ZOE       |ALBANY     |F  |5    |
|2007|ZISSY     |ROCKLAND   |F  |5    |
|2007|ZISSY     |KINGS      |F  |27   |
|2007|ZION      |KINGS      |M  |15   |
|2007|ZION      |BRONX      |M  |14   |
|2007|ZEV       |ROCKLAND   |M  |6    |
+----+----------+-----------+---+-----+
only showing top 20 rows



In [9]:
df.rdd.getNumPartitions()

1

In [14]:
# groupBy by partitionId
from pyspark.sql.functions import spark_partition_id, asc, desc

df3 = (
    df2.withColumn("partitionId", spark_partition_id())
    .groupBy("partitionId")
    .count()
    .orderBy(desc("count"))
)
df3.show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0| 5226|
|          9| 5226|
|          1| 5225|
|          2| 5225|
|          3| 5225|
|          4| 5225|
|          5| 5225|
|          6| 5225|
|          7| 5225|
|          8| 5225|
+-----------+-----+



In [19]:
df.describe().show(truncate=False)

+-------+------------------+----------+------+-----+-----------------+
|summary|Year              |First Name|County|Sex  |Count            |
+-------+------------------+----------+------+-----+-----------------+
|count  |52252             |52252     |52252 |52252|52252            |
|mean   |2010.6141583097299|NULL      |NULL  |NULL |18.05534716374493|
|stddev |2.355530993500322 |NULL      |NULL  |NULL |22.67625097645675|
|min    |2007              |AADEN     |ALBANY|F    |5                |
|max    |2014              |ZURI      |YATES |M    |297              |
+-------+------------------+----------+------+-----+-----------------+



In [21]:
df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- First Name: string (nullable = true)
 |-- County: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Count: integer (nullable = true)

